In [22]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from sklearn.model_selection import train_test_split
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [23]:
import glob
files = glob.glob("../../datasets/camel-1.2.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [24]:
ant_master

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,camel,1.2,org.apache.camel.component.irc.IrcErrorLogger,9,0,0,5,24,0,1,...,1.0,1,0.000000,0.370370,0,0,22.555556,1,0.8889,0
1,camel,1.2,org.apache.camel.ProducerTemplate,27,1,0,6,27,351,1,...,0.0,0,0.000000,0.412037,0,0,0.000000,1,1.0000,3
2,camel,1.2,org.apache.camel.impl.JndiRegistry,9,1,0,3,18,30,1,...,1.0,0,0.000000,0.355556,0,0,8.000000,1,0.7778,0
3,camel,1.2,org.apache.camel.component.cxf.CxfSoapConsumer,7,1,0,27,50,7,3,...,1.0,3,0.000000,0.416667,0,0,26.428571,1,0.7143,0
4,camel,1.2,org.apache.camel.processor.TryProcessor,8,2,0,10,28,0,1,...,1.0,3,0.684211,0.371429,1,1,26.500000,2,1.0000,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,camel,1.2,org.apache.camel.builder.PredicateFactory,1,1,0,1,1,0,0,...,0.0,0,0.000000,1.000000,0,0,0.000000,1,1.0000,1
604,camel,1.2,org.apache.camel.processor.RedeliveryPolicy,26,1,0,5,38,251,5,...,1.0,0,0.000000,0.292308,0,0,8.000000,6,1.2308,0
605,camel,1.2,org.apache.camel.model.NodeFactory,3,1,0,3,6,3,1,...,0.0,0,0.000000,1.000000,0,0,3.666667,1,0.6667,0
606,camel,1.2,org.apache.camel.spring.SpringInstrumentationA...,4,3,0,6,13,4,1,...,1.0,1,0.939394,1.000000,0,0,5.500000,1,0.5000,0


In [25]:
train, test = train_test_split(ant_master, test_size=0.2)

In [26]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [27]:
x_train

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
98,6,1,0,9,18,15,8,1,5,2.000000,102,0.0,0,0.000000,0.333333,0,0,16.000000,5,2.8333
475,43,1,0,9,100,825,4,7,35,0.838095,785,0.8,1,0.000000,0.261905,0,0,17.023256,1,0.8372
135,0,1,0,0,0,0,0,0,0,2.000000,0,0.0,0,0.000000,0.000000,0,0,0.000000,0,0.0000
97,15,2,0,18,28,101,5,17,13,0.750000,118,1.0,2,0.548387,0.200000,1,1,6.733333,2,1.0667
2,9,1,0,3,18,30,1,2,8,0.375000,82,1.0,0,0.000000,0.355556,0,0,8.000000,1,0.7778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,21,2,1,15,49,142,1,14,12,0.808333,276,1.0,2,0.400000,0.200000,1,2,11.857143,3,1.0000
193,39,1,11,23,67,687,13,10,31,0.897368,405,1.0,7,0.000000,0.133772,0,0,9.128205,4,1.3846
570,2,1,0,10,12,0,2,8,2,0.000000,49,1.0,2,0.000000,0.583333,0,0,22.500000,1,0.5000
339,9,2,1,7,18,0,1,6,7,0.250000,66,1.0,1,0.650000,0.388889,1,1,6.222222,1,0.7778


In [28]:
x_test

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
35,20,1,2,101,54,168,98,5,12,0.954887,352,0.285714,0,0.000000,0.219298,0,0,16.250000,3,1.1500
524,4,3,0,9,15,4,0,9,1,0.333333,62,1.000000,1,0.920000,0.666667,1,1,14.250000,4,1.2500
75,1,2,0,1,2,0,0,1,1,2.000000,6,1.000000,0,1.000000,1.000000,0,0,3.000000,0,0.0000
369,12,1,0,14,24,66,1,13,11,2.000000,74,0.000000,0,0.000000,0.444444,0,0,5.166667,1,0.9167
150,0,1,0,0,0,0,0,0,0,2.000000,0,0.000000,0,0.000000,0.000000,0,0,0.000000,0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,2,5,0,2,6,0,1,1,2,1.000000,22,1.000000,0,0.944444,0.666667,0,0,9.000000,1,0.5000
274,2,1,0,5,6,0,0,5,2,0.000000,30,1.000000,2,0.000000,0.625000,0,0,13.000000,1,0.5000
257,20,3,0,18,53,126,4,17,14,0.782895,299,1.000000,2,0.714286,0.160000,1,1,13.550000,2,1.0500
245,4,2,0,2,6,4,1,1,4,0.333333,24,1.000000,0,0.857143,0.750000,1,2,4.750000,1,0.5000


In [29]:
y_train = train['bug']
y_test = test['bug']

In [30]:
y_train

98     3
475    2
135    0
97     0
2      0
      ..
200    0
193    4
570    0
339    1
587    3
Name: bug, Length: 486, dtype: int64

In [31]:
y_test

35     0
524    2
75     0
369    1
150    0
      ..
276    0
274    0
257    0
245    0
160    0
Name: bug, Length: 122, dtype: int64

In [32]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [33]:
x_train_np

array([[ 6.        ,  1.        ,  0.        , ..., 16.        ,
         5.        ,  2.8333    ],
       [43.        ,  1.        ,  0.        , ..., 17.02325581,
         1.        ,  0.8372    ],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 2.        ,  1.        ,  0.        , ..., 22.5       ,
         1.        ,  0.5       ],
       [ 9.        ,  2.        ,  1.        , ...,  6.22222222,
         1.        ,  0.7778    ],
       [48.        ,  1.        ,  0.        , ..., 14.08333333,
         2.        ,  1.        ]])

In [34]:
x_test_np

array([[20.        ,  1.        ,  2.        , ..., 16.25      ,
         3.        ,  1.15      ],
       [ 4.        ,  3.        ,  0.        , ..., 14.25      ,
         4.        ,  1.25      ],
       [ 1.        ,  2.        ,  0.        , ...,  3.        ,
         0.        ,  0.        ],
       ...,
       [20.        ,  3.        ,  0.        , ..., 13.55      ,
         2.        ,  1.05      ],
       [ 4.        ,  2.        ,  0.        , ...,  4.75      ,
         1.        ,  0.5       ],
       [11.        ,  1.        ,  0.        , ..., 21.45454545,
        10.        ,  1.9091    ]])

In [35]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [36]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [37]:
train_x

array([[[ 6.        ,  1.        ,  0.        , ..., 16.        ,
          5.        ,  2.8333    ]],

       [[43.        ,  1.        ,  0.        , ..., 17.02325581,
          1.        ,  0.8372    ]],

       [[ 0.        ,  1.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       ...,

       [[ 2.        ,  1.        ,  0.        , ..., 22.5       ,
          1.        ,  0.5       ]],

       [[ 9.        ,  2.        ,  1.        , ...,  6.22222222,
          1.        ,  0.7778    ]],

       [[48.        ,  1.        ,  0.        , ..., 14.08333333,
          2.        ,  1.        ]]])

In [38]:
test_y

array([0, 2, 0, 1, 0, 0, 0, 3, 2, 3, 1, 1, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0,
       0, 1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 3, 1, 4, 1, 0, 2, 0, 1, 0, 0, 0,
       0, 5, 0, 1, 0, 3, 0, 0, 2, 2, 0, 0, 0, 0, 5, 1, 0, 0, 1, 0, 4, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       2, 0, 4, 1, 0, 0, 1, 1, 0, 1, 0, 0, 8, 2, 1, 0, 0, 0, 0, 0, 0, 2,
       0, 2, 0, 5, 0, 1, 0, 0, 0, 0, 0, 0])

In [39]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [40]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, callbacks=[tensorboard_callback])

Epoch 1/100
16/16 [==============================] - 5s 25ms/step - loss: 4.7822 - mse: 4.7822 - mae: 0.8786 - root_mean_squared_error: 2.1868 - mean_squared_logarithmic_error: 0.5043
Epoch 2/100
16/16 [==============================] - 0s 6ms/step - loss: 4.7604 - mse: 4.7604 - mae: 0.8831 - root_mean_squared_error: 2.1818 - mean_squared_logarithmic_error: 0.4946
Epoch 3/100
16/16 [==============================] - 0s 6ms/step - loss: 4.7324 - mse: 4.7324 - mae: 0.8875 - root_mean_squared_error: 2.1754 - mean_squared_logarithmic_error: 0.4827
Epoch 4/100
16/16 [==============================] - 0s 6ms/step - loss: 4.6954 - mse: 4.6954 - mae: 0.8938 - root_mean_squared_error: 2.1669 - mean_squared_logarithmic_error: 0.4687
Epoch 5/100
16/16 [==============================] - 0s 6ms/step - loss: 4.6422 - mse: 4.6422 - mae: 0.9043 - root_mean_squared_error: 2.1546 - mean_squared_logarithmic_error: 0.4495
Epoch 6/100
16/16 [==============================] - 0s 6ms/step - loss: 4.5640 - ms

16/16 [==============================] - 0s 5ms/step - loss: 3.3313 - mse: 3.3313 - mae: 0.9893 - root_mean_squared_error: 1.8252 - mean_squared_logarithmic_error: 0.3189
Epoch 45/100
16/16 [==============================] - 0s 5ms/step - loss: 3.3284 - mse: 3.3284 - mae: 0.9950 - root_mean_squared_error: 1.8244 - mean_squared_logarithmic_error: 0.3221
Epoch 46/100
16/16 [==============================] - 0s 5ms/step - loss: 3.2926 - mse: 3.2926 - mae: 0.9991 - root_mean_squared_error: 1.8145 - mean_squared_logarithmic_error: 0.3251
Epoch 47/100
16/16 [==============================] - 0s 5ms/step - loss: 3.2870 - mse: 3.2870 - mae: 0.9729 - root_mean_squared_error: 1.8130 - mean_squared_logarithmic_error: 0.3107
Epoch 48/100
16/16 [==============================] - 0s 5ms/step - loss: 3.2654 - mse: 3.2654 - mae: 0.9963 - root_mean_squared_error: 1.8071 - mean_squared_logarithmic_error: 0.3264
Epoch 49/100
16/16 [==============================] - 0s 5ms/step - loss: 3.2461 - mse: 3.246

16/16 [==============================] - 0s 5ms/step - loss: 2.6572 - mse: 2.6572 - mae: 0.7969 - root_mean_squared_error: 1.6301 - mean_squared_logarithmic_error: 0.2368
Epoch 89/100
16/16 [==============================] - 0s 5ms/step - loss: 2.6452 - mse: 2.6452 - mae: 0.8243 - root_mean_squared_error: 1.6264 - mean_squared_logarithmic_error: 0.2508
Epoch 90/100
16/16 [==============================] - 0s 5ms/step - loss: 2.6071 - mse: 2.6071 - mae: 0.8193 - root_mean_squared_error: 1.6147 - mean_squared_logarithmic_error: 0.2477
Epoch 91/100
16/16 [==============================] - 0s 6ms/step - loss: 2.5973 - mse: 2.5973 - mae: 0.7932 - root_mean_squared_error: 1.6116 - mean_squared_logarithmic_error: 0.2332
Epoch 92/100
16/16 [==============================] - 0s 5ms/step - loss: 2.6015 - mse: 2.6015 - mae: 0.7920 - root_mean_squared_error: 1.6129 - mean_squared_logarithmic_error: 0.2345
Epoch 93/100
16/16 [==============================] - 0s 5ms/step - loss: 2.5961 - mse: 2.596

In [41]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 3750), started 3:29:37 ago. (Use '!kill 3750' to kill it.)

In [42]:
score = model.evaluate(test_x, test_y)
score

4/4 [==============================] - 1s 3ms/step - loss: 1.7255 - mse: 1.7255 - mae: 0.8611 - root_mean_squared_error: 1.3136 - mean_squared_logarithmic_error: 0.3285


[1.7255321741104126,
 1.7255321741104126,
 0.8611381649971008,
 1.313595175743103,
 0.32846319675445557]